Tuesday, November 14, 2023

https://www.youtube.com/watch?v=wgYctKFnQ74

So with a few minor tweaks I got this to work! Nice!

Thanks to the starter code - https://huggingface.co/spaces/PyaeSoneK/chatchat


In [1]:
!nvidia-smi

Tue Nov 14 11:07:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   55C    P0              N/A /  70W |    505MiB /  2048MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Install the libraries

In [ ]:
# !pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.6 MB/s eta 0:00:00


Load the libraries

In [1]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap


Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [2]:
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
# model = AutoModelForCausalLM.from_pretrained(checkpoint,
#                                              device_map='auto',
#                                              torch_dtype=torch.float16,
#                                              load_in_4bit=True,
#                                              bnb_4bit_quant_type="nf4",
#                                              bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define Transformers Pipeline which will be fed into Langchain

In [4]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [5]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced Life guru and mental health expert that excels at giving advice.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


All the helper fucntions to generate prompt, prompt template, clean up output text

In [6]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [7]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})



In [8]:
system_prompt = "You are an advanced Life guru and mental health expert that excels at giving advice. "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [9]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)



In [10]:
text = "My life sucks, what do you suggest? Please don't tell me to medidate"

In [11]:
response = llm_chain.run(text)
print(response)

  Ah, I see. Well, my dear, let me tell you something – life can indeed be challenging at times, but that's exactly why we need to take a step back and reassess our priorities. Meditation is a wonderful tool, but it's not the only solution to your problems. Here's a well-reasoned and step-by-step approach to help you navigate your current situation:

Step 1: Identify the Root of the Problem

* Take a moment to reflect on what's causing your dissatisfaction. Is it a toxic relationship? A lack of purpose or meaning? Unresolved emotional trauma? Once you've identified the root of the problem, you can start working on a solution.

Step 2: Seek Professional Help

* Sometimes, we need outside help to overcome our challenges. Consider consulting a therapist or counselor who can provide you with emotional support and guidance. They can help you develop coping strategies and work through any underlying issues that may be contributing to your unhappiness.

Step 3: Practice Self-Care

* Taking ca